## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel]"
!python -c "import matplotlib" || pip install -q matplotlib
!pip install -q pytorch-lightning~=2.0
%matplotlib inline

/bin/bash: line 1: python: command not found
/bin/bash: line 1: python: command not found


/home/gasyna/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## Setup imports

In [1]:
import pytorch_lightning
from pytorch_lightning.callbacks import ModelCheckpoint
from monai.utils import set_determinism
from monai.transforms import (
AsDiscrete,
EnsureChannelFirstd,
Compose,
CropForegroundd,
LoadImaged,
Orientationd,
RandCropByPosNegLabeld,
ScaleIntensityRanged,
Spacingd,
EnsureType,
EnsureTyped,
)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, list_data_collate, decollate_batch, DataLoader
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob
import nibabel as nib
import numpy as np
from natsort import natsorted
from sklearn.model_selection import KFold


print_config()

/home/gasyna/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2024-06-27 21:40:03.997901: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


MONAI version: 1.4.dev2425
Numpy version: 1.23.5
Pytorch version: 2.0.0+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: b5d71491b839191430b236bd3b233c5f29daf229
MONAI __file__: /home/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.2.1
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.8.0
Pillow version: 9.0.1
Tensorboard version: 2.12.2
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.1+cu117
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.7
pandas version: 1.5.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INST

## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp/tmpqm2bg8jv


In [3]:
data_dir = os.path.join(os.getcwd(), 'AeroPath')

## Download dataset

Downloads and extracts the dataset

In [4]:
resource = "https://zenodo.org/records/10069289/files/AeroPath.zip?download=1"
md5 = "3fd5106c175c85d60eaece220f5dfd87"

compressed_file = os.path.join(root_dir, "AeroPath.zip")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir, md5)

## Define the LightningModule

The LightningModule contains a refactoring of your training code. The following module is a refactoring of the code in `spleen_segmentation_3d.ipynb`:

In [5]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_dice',
    dirpath=os.path.join(root_dir, 'checkpoints'),  # Directory to save checkpoints
    filename='whole_clipped',  # Filename prefix for saving checkpoints
    save_top_k=1,  # Save only the best checkpoint
    mode='max',  # `min` for minimizing the metric, `max` for maximizing
    verbose=True,  # Log a message when saving the best checkpoint
)

In [6]:
class Net(pytorch_lightning.LightningModule):
    def __init__(self):
        super().__init__()
        self._model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=2,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH,
        )
        self.loss_function = DiceLoss(to_onehot_y=True, softmax=True)
        self.post_pred = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(argmax=True, to_onehot=2)])
        self.post_label = Compose([EnsureType("tensor", device="cpu"), AsDiscrete(to_onehot=2)])
        self.dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
        self.best_val_dice = 0
        self.best_val_epoch = 0
        self.validation_step_outputs = []


        self.common_transforms = Compose(
        [
            LoadImaged(keys=["image", "label"]),
            EnsureChannelFirstd(keys=["image", "label"]),
            Orientationd(keys=["image", "label"], axcodes="RAS"),
            Spacingd(
                keys=["image", "label"],
                pixdim=(1.5, 1.5, 2.0),
                mode=("bilinear", "nearest"),
            ),
            ScaleIntensityRanged(
                keys=["image"],
                a_min=-1024,
                a_max=1024,
                b_min=0.0,
                b_max=1.0,
                clip=True,
            ),
            CropForegroundd(keys=["image", "label"], source_key="image"),
            EnsureTyped(keys=["image", "label"]),
        ]
        )


    def forward(self, x):
        return self._model(x)

    def prepare_data(self):
        # # set up the correct data path
        # pattern = os.path.join(data_dir, '**/*_CT_HR_label_airways.nii.gz')
        # train_labels = sorted(glob.glob(pattern, recursive=True))

        # pattern = os.path.join(data_dir, '**/*_CT_HR.nii.gz')
        # train_images = sorted(glob.glob(pattern, recursive=True))

        pattern = os.path.join('nonoverlapping_labels', '**/quadrant_1_*.nii.gz')
        train_labels = sorted(glob.glob(pattern, recursive=True))

        pattern = os.path.join('nonoverlapping_quadrants', '**/quadrant_1_*_CT_HR.nii.gz')
        train_images = sorted(glob.glob(pattern, recursive=True))

        data_dicts = [
            {"image": image_name, "label": label_name} for image_name, label_name in zip(train_images, train_labels)
        ]
        train_files, val_files = data_dicts[:-9], data_dicts[-9:]

        # set deterministic training for reproducibility
        set_determinism(seed=0)

        # define the data transforms
        train_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                EnsureChannelFirstd(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    pixdim=(1.5, 1.5, 2.0),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"],
                    a_min=-1024,
                    a_max=1024,
                    b_min=0.0,
                    b_max=1.0,
                    clip=True,
                ),
                CropForegroundd(keys=["image", "label"], source_key="image"),
                # randomly crop out patch samples from
                # big image based on pos / neg ratio
                # the image centers of negative samples
                # must be in valid image area

                RandCropByPosNegLabeld(
                    keys=["image", "label"],
                    label_key="label",
                    spatial_size=(64, 64, 64),
                    pos=1,
                    neg=1,
                    num_samples=4,
                    image_key="image",
                    image_threshold=0,
                ),

                # user can also add other random transforms
                #                 RandAffined(
                #                     keys=['image', 'label'],
                #                     mode=('bilinear', 'nearest'),
                #                     prob=1.0,
                #                     spatial_size=(96, 96, 96),
                #                     rotate_range=(0, 0, np.pi/15),
                #                     scale_range=(0.1, 0.1, 0.1)),
            ]
        )
        val_transforms = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                EnsureChannelFirstd(keys=["image", "label"]),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    pixdim=(1.5, 1.5, 2.0),
                    mode=("bilinear", "nearest"),
                ),
                ScaleIntensityRanged(
                    keys=["image"],
                    a_min=-1024,
                    a_max=1024,
                    b_min=0.0,
                    b_max=1.0,
                    clip=True,
                ),
                CropForegroundd(keys=["image", "label"], source_key="image"),
            ]
        )
                    

        # we use cached datasets - these are 10x faster than regular datasets
        self.train_ds = CacheDataset(
            data=train_files,
            transform=train_transforms,
            cache_rate=1.0,
            num_workers=4,
        )
        self.val_ds = CacheDataset(
            data=val_files,
            transform=val_transforms,
            cache_rate=1.0,
            num_workers=4,
        )


    def train_dataloader(self):
        train_loader = DataLoader(
            self.train_ds,
            batch_size=2,
            shuffle=True,
            num_workers=4,
            collate_fn=list_data_collate,
        )
        return train_loader

    def val_dataloader(self):
        val_loader = DataLoader(self.val_ds, batch_size=1, num_workers=4)
        return val_loader

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self._model.parameters(), 1e-4)
        return optimizer

    def training_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        output = self.forward(images)
        loss = self.loss_function(output, labels)
        tensorboard_logs = {"train_loss": loss.item()}
        return {"loss": loss, "log": tensorboard_logs}

    
    def validation_step(self, batch, batch_idx):
        images, labels = batch["image"], batch["label"]
        roi_size = (160, 160, 160)
        sw_batch_size = 4
        outputs = sliding_window_inference(images, roi_size, sw_batch_size, self)
        loss = self.loss_function(outputs, labels)
        outputs = [self.post_pred(i) for i in decollate_batch(outputs)]
        labels = [self.post_label(i) for i in decollate_batch(labels)]
        self.dice_metric(y_pred=outputs, y=labels)
        d = {"val_loss": loss, "val_number": len(outputs)}
        self.validation_step_outputs.append(d)
        return d
    


    
    def perform_inference(self, model, data):
        # Perform inference using the model
        with torch.no_grad():
            data = torch.DoubleTensor(data)  # Convert data to type Double
            model_output = model(data.unsqueeze(0))
        return model_output

    def on_validation_epoch_end(self):
        val_loss, num_items = 0, 0
        for output in self.validation_step_outputs:
            val_loss += output["val_loss"].sum().item()
            num_items += output["val_number"]
        mean_val_dice = self.dice_metric.aggregate().item()
        self.dice_metric.reset()
        mean_val_loss = torch.tensor(val_loss / num_items)
        tensorboard_logs = {
            "val_dice": mean_val_dice,
            "val_loss": mean_val_loss,
        }
        if mean_val_dice > self.best_val_dice:
            self.best_val_dice = mean_val_dice
            self.best_val_epoch = self.current_epoch
        print(
            f"current epoch: {self.current_epoch} "
            f"current mean dice: {mean_val_dice:.4f}"
            f"\nbest mean dice: {self.best_val_dice:.4f} "
            f"at epoch: {self.best_val_epoch}"
        )
        self.validation_step_outputs.clear()  # free memory
        self.log('val_dice', mean_val_dice, on_step=False, on_epoch=True, prog_bar=True, logger=True) # log

        return {"log": tensorboard_logs}
    
    def infer_on_single_image(self, data):
        self._model.eval()
        with torch.no_grad():
            roi_size = (64, 64, 64)
            sw_batch_size = 4
            outputs = sliding_window_inference(data, roi_size, sw_batch_size, self._model)
            post_processed_outputs = [self.post_pred(i) for i in decollate_batch(outputs)]
        return post_processed_outputs
    

    def infer(self, file_path, label_path = None):
        self._model.eval()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self._model.to(device)

        data_dict = {"image": file_path, "label": label_path}

        # Apply transformations
        data_dict = self.common_transforms(data_dict)
        data = data_dict["image"]
        label = data_dict["label"]

        # Convert data and label to tensors and add batch dimension

        data = torch.tensor(data).unsqueeze(0).to(device)
        label = torch.tensor(label).unsqueeze(0).to(device)


        # Run inference
        predictions = self.infer_on_single_image(data)

        # Process and save the predictions as needed
        for i, prediction in enumerate(predictions):
            # Save or process each prediction here

            prediction_np = prediction.cpu().numpy()
            # Save as NIfTI file, for testing only
            pred_img = nib.Nifti1Image(prediction_np, nib.load(file_path).affine)

        # Convert predictions and labels to binary format if necessary
        post_pred = Compose([AsDiscrete(argmax=True, to_onehot=2)])
        post_label = Compose([AsDiscrete(to_onehot=2)])

        # Apply the transformations directly to the tensors and ensure they are on the same device
        prediction_tensor = post_pred(prediction.to(device))
        label_tensor = post_label(label.to(device))

        dice_metric = DiceMetric(include_background=True, reduction="mean")
        dice_metric(y_pred=prediction_tensor, y=label_tensor)
        dice_score = dice_metric.aggregate().item()
        dice_metric.reset()

        print(f"Dice Score: {dice_score:.4f}")
        print("Inference complete.")

        return prediction_tensor.cpu(), label_tensor.cpu()
    
    def dice_score(self, prediction_tensor, label_tensor):
        # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # post_pred = Compose([AsDiscrete(argmax=True, to_onehot=2)])
        # post_label = Compose([AsDiscrete(to_onehot=2)])

        # prediction_tensor = post_pred(prediction.to(device))
        # label_tensor = post_label(label.to(device))

        # Compute Dice score
        dice_metric = DiceMetric(include_background=True, reduction="mean")
        dice_metric(y_pred=prediction_tensor, y=label_tensor)
        dice_score = dice_metric.aggregate().item()
        dice_metric.reset()

        print(dice_score)




In [ ]:

pattern = os.path.join('overlapping_labels', '**/quadrant_1_*.nii.gz')
train_labels = sorted(glob.glob(pattern, recursive=True))

pattern = os.path.join('overlapping_quadrants', '**/quadrant_1_*_CT_HR.nii.gz')
train_images = sorted(glob.glob(pattern, recursive=True))

In [7]:
# Check dimensions of random sample

img = 'nonoverlapping_quadrants/24/quadrant_2_7_CT_HR.nii.gz'
label = 'nonoverlapping_labels/24/quadrant_2_7_CT_HR_label_airways.nii.gz'
img = nib.load(img).get_fdata()
label = nib.load(label).get_fdata()

print(f'img shape: {img.shape}, label shape: {label.shape}')

img shape: (512, 512, 362), label shape: (512, 512, 362)


## Run the training

In [13]:
# initialise the LightningModule
net = Net()

# set up loggers and checkpoints
log_dir = os.path.join(root_dir, "logs")
tb_logger = pytorch_lightning.loggers.TensorBoardLogger(save_dir=log_dir)

# initialise Lightning's trainer.
trainer = pytorch_lightning.Trainer(
    devices=[0],
    max_epochs=600,
    logger=tb_logger,
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=1,
    log_every_n_steps=16,
)

# train
trainer.fit(net)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gasyna/.local/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
Loading dataset: 100%|██████████| 9/9 [00:42<00:00,  4.73s/it]
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /tmp/tm

Sanity Checking: 0it [00:00, ?it/s]

current epoch: 0 current mean dice: 0.0000
best mean dice: 0.0000 at epoch: 0


/home/gasyna/.local/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=16). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Epoch 598, global step 5391: 'val_dice' was not in top 1


current epoch: 598 current mean dice: 0.8105
best mean dice: 0.8190 at epoch: 597


Validation: 0it [00:00, ?it/s]

Epoch 599, global step 5400: 'val_dice' was not in top 1
`Trainer.fit` stopped: `max_epochs=600` reached.


current epoch: 599 current mean dice: 0.7879
best mean dice: 0.8190 at epoch: 597


In [18]:
# initialise the LightningModule
net = Net()

# set up loggers and checkpoints
log_dir = os.path.join(root_dir, "logs")
tb_logger = pytorch_lightning.loggers.TensorBoardLogger(save_dir=log_dir)

# initialise Lightning's trainer.
trainer = pytorch_lightning.Trainer(
    devices=[0],
    max_epochs=600,
    logger=tb_logger,
    enable_checkpointing=True,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=1,
    log_every_n_steps=16,
)

# train
trainer.fit(net)

/home/gasyna/.local/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
print(f"train completed, best_metric: {net.best_val_dice:.4f} " f"at epoch {net.best_val_epoch}")

train completed, best_metric: 0.8190 at epoch 597


In [21]:
net2 = net  

In [22]:
net.prepare_data()

/home/gasyna/.local/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
Loading dataset: 100%|██████████| 9/9 [00:33<00:00,  3.68s/it]


In [ ]:
import torch
from monai.networks.nets import UNet


# Load the model weights from the checkpoint file
checkpoint_path = 'best-checkpoint.ckpt'
model = Net.load_from_checkpoint(checkpoint_path)

# Set the model to evaluation mode
model.eval()


## Model Ensembling

## View training in tensorboard

Please uncomment the following cell to load tensorboard results.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=$log_dir

## Load model and create prediction files

In [16]:
net = Net.load_from_checkpoint('checkpoints/best-checkpoint_whole_64_0.8023 at epoch: 427.ckpt', strict=False)
# net = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_1Q_spatial_size_64_0.6679 at epoch: 498.ckpt')
# net = Net.load_from_checkpoint('checkpoints/best-checkpoint_new_method_2Q_spatial_size_64_0.8143 at epoch 499.ckpt')

# net = Net.load_from_checkpoint('whole_clipped2.ckpt', strict=False)
net.prepare_data()

/home/gasyna/.local/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/home/gasyna/.local/lib/python3.10/site-packages/pytorch_lightning/core/saving.py:162: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['loss_function.class_weight']
  rank_zero_warn(
Loading dataset: 100%|██████████| 9/9 [00:30<00:00,  3.35s/it]


In [17]:
# model = Net.load_from_checkpoint('checkpoints/best-checkpoint_whole_64_0.8023 at epoch: 427.ckpt')

# model.eval()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)

# if net is None:
#     net = Net.load_from_checkpoint('checkpoints/best-checkpoint_whole_64_0.8023 at epoch: 427.ckpt')
#     net.prepare_data()

net.eval()
device = torch.device("cuda:0")
net.to(device)
with torch.no_grad():
    for i, val_data in enumerate(net.val_dataloader()):
        roi_size = (64, 64, 64)
        sw_batch_size = 4
        val_outputs = sliding_window_inference(val_data["image"].to(device), roi_size, sw_batch_size, net)
        # plot the slice [:, :, 80]
        # plt.figure("check", (18, 6))
        # plt.subplot(1, 3, 1)
        # plt.title(f"image {i}")
        # plt.imshow(val_data["image"][0, 0, :, :, 80], cmap="gray")
        # plt.subplot(1, 3, 2)
        # plt.title(f"label {i}")
        # plt.imshow(val_data["label"][0, 0, :, :, 80])
        # plt.subplot(1, 3, 3)
        # plt.title(f"output {i}")
        pred = torch.argmax(val_outputs, dim=1).detach().cpu()
        # plt.imshow(pred[0, :, :, 80])
        plt.show()


        pred_np = pred.cpu().numpy()[0, :, :, :]

        dir_name = 'whole'
        # dir_name = '1Q'
        # dir_name = '2Q'

        nib.save(nib.Nifti1Image(pred_np.astype(float), nib.load('AeroPath/1/1_CT_HR_label_airways.nii.gz').affine), f'pred/{dir_name}/{i}.nii.gz')

        # label = val_data['label'].cpu().numpy()[0, 0, :, :, :]
        # nib.save(nib.Nifti1Image(label.astype(float), nib.load('AeroPath/1/1_CT_HR_label_airways.nii.gz').affine), f'labels/{i}.nii.gz')





## Interpolate, Concat and ensemble predictions

In [24]:
import numpy as np
import torch
import torch.nn.functional as F
import nibabel as nib
from monai.transforms import Compose, AsDiscrete
from monai.metrics import DiceMetric

def interpolate_predictions(predictions, target_shape):
    # Ensure predictions tensor has 5 dimensions: (N, C, D, H, W)
    predictions = torch.tensor(predictions).unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

    # Interpolate the predictions using nearest neighbor interpolation
    interpolated_predictions = F.interpolate(predictions, size=target_shape, mode='nearest')
    
    # Remove batch and channel dimensions after interpolation
    interpolated_predictions = interpolated_predictions.squeeze(0).squeeze(0).numpy()
    
    return interpolated_predictions

# Function to calculate Dice score
def calc_dice_score(pred, label):
    post_pred = Compose([AsDiscrete(argmax=False, to_onehot=2)])
    post_label = Compose([AsDiscrete(to_onehot=2)])

    # Ensure the inputs have the correct dimensions
    pred = torch.tensor(pred).unsqueeze(0).unsqueeze(0)
    label = torch.tensor(label).unsqueeze(0).unsqueeze(0)

    prediction_tensor = post_pred(pred)
    label_tensor = post_label(label)

    # Compute Dice score
    dice_metric = DiceMetric(include_background=False, reduction="mean")
    dice_metric(y_pred=prediction_tensor, y=label_tensor)

    dice_score = dice_metric.aggregate().item()
    dice_metric.reset()

    return dice_score


labels = natsorted(glob.glob('labels/*', recursive=True))


# Example usage with NIfTI files

for idx, _ in enumerate(labels):
    label = nib.load(f'labels/{idx}.nii.gz').get_fdata()
    whole = nib.load(f'pred/whole/{idx}.nii.gz').get_fdata()
    pred_1Q = nib.load(f'pred/1Q/{idx}.nii.gz').get_fdata()
    pred_2Q = nib.load(f'pred/2Q/{idx}.nii.gz').get_fdata()


    # Interpolate predictions to allow for concatenation

    inter_pred_1Q = interpolate_predictions(pred_1Q, (*whole.shape[:-1], pred_1Q.shape[-1]))
    inter_pred_2Q = interpolate_predictions(pred_2Q, (*whole.shape[:-1], pred_2Q.shape[-1]))


    # Merge interpolated predictions along the last axis (z-axis)
    merged = np.concatenate((inter_pred_1Q, inter_pred_2Q), axis=2)

    # Interpolate predictions to match whole.shape

    merged = interpolate_predictions(merged, whole.shape)

    # Save the merged predictions as a NIfTI file
    merged_nifti = nib.Nifti1Image(merged, nib.load('AeroPath/1/1_CT_HR.nii.gz').affine)
    nib.save(merged_nifti, 'pred_test_merged.nii.gz')
    # print(f'Merged shape: {merged.shape}')

    # Perform ensembling using the maximum values
    ensembled = np.maximum(merged, whole)

    # Save the ensembled predictions as a NIfTI file
    ensembled_nifti = nib.Nifti1Image(ensembled, nib.load('AeroPath/1/1_CT_HR.nii.gz').affine)
    nib.save(ensembled_nifti, f'pred/ensembled/{idx}.nii.gz')

    # Compute Dice score
    dice_whole = calc_dice_score(whole, label)
    print(f'Dice Score for whole prediction: {dice_whole:.4f}')

    dice_ensembled = calc_dice_score(ensembled, label)
    print(f'Dice Score for ensembled prediction: {dice_ensembled:.4f}')

    print('*'*50)



RuntimeError: The size of tensor a (267) must match the size of tensor b (182) at non-singleton dimension 1

In [18]:
ensembled_nifti = nib.Nifti1Image(ensembled, nib.load('AeroPath/1/1_CT_HR.nii.gz').affine)
nib.save(ensembled_nifti, f'pred/ensembled/{idx}.nii.gz')

# Compute Dice score
dice_whole = calc_dice_score(whole, label)
print(f'Dice Score for whole prediction: {dice_whole:.4f}')

dice_ensembled = calc_dice_score(ensembled, label)
print(f'Dice Score for ensembled prediction: {dice_ensembled:.4f}')

Dice Score for whole prediction: 0.9051
Dice Score for ensembled prediction: 0.8460


In [21]:
def calc_dice_score(pred, label):
    post_pred = Compose([AsDiscrete(argmax=False, to_onehot=2)])
    post_label = Compose([AsDiscrete(to_onehot=2)])

    # Ensure the inputs have the correct dimensions
    pred = torch.tensor(pred).unsqueeze(0).unsqueeze(0)
    label = torch.tensor(label).unsqueeze(0).unsqueeze(0)

    prediction_tensor = post_pred(pred)
    label_tensor = post_label(label)

    # Compute Dice score
    dice_metric = DiceMetric(include_background=False, reduction="mean")
    dice_metric(y_pred=prediction_tensor, y=label_tensor)

    dice_score = dice_metric.aggregate().item()
    dice_metric.reset()

    return dice_score

label = val_data['label'].cpu().numpy()[0, 0, :, :, :]
dice_whole = calc_dice_score(pred, label)
print(f'Dice Score for whole prediction: {dice_whole:.4f}')

/tmp/ipykernel_7544/2687408277.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = torch.tensor(pred).unsqueeze(0).unsqueeze(0)


Dice Score for whole prediction: 0.4997


## Cleanup data directory

Remove directory if a temporary was used.

In [ ]:
if directory is None:
    shutil.rmtree(root_dir)